In [6]:
import numpy as np
import sklearn.metrics as metrics
import spectral_cluster


def StochasticBlockModel(n, k, p, q):
    sizes = np.full(k, n)
    prob_matrix = np.full((k, k), q)
    np.fill_diagonal(prob_matrix,  p)
    return spectral_cluster.generate_sbm(sizes, prob_matrix)


def Measurements(n, k, iteration=10):
    """It outputs a 5x5 matrix filled with adjusted Rand scores as follows. For p in [0.6,0.7,0.8,0.9,1] and
    q in [0.1,0.2,0.3,0.4,0.5] it creates a Stochastic Block Graph with parameters n,k,p,q, then apply the
    Spectral Clustering and finally compare the true labels with the obtained ones via adjusted Rand
    score. We do it iteration times for fixed p,q and take the avegare of these values."""
    s = iteration
    M = np.zeros((5, 5))
    P = [0.6, 0.7, 0.8, 0.9, 1]
    Q = [0.1, 0.2, 0.3, 0.4, 0.5]
    for p in P:
        for q in Q:
            list_pq = []
            for _ in range(s):
                labels_true, G = StochasticBlockModel(n, k, p, q)
                try:
                    labels_pred = spectral_cluster.solve(G, k)
                    a = metrics.adjusted_rand_score(labels_true, labels_pred)
                    list_pq.append(a)
                except spectral_cluster.ClusterError:
                    list_pq.append(0)
            M[P.index(p), Q.index(q)] = sum(list_pq)/s

    M1 = np.round(M, 3)
    return M1

In [7]:
Measurements(40, 3, 10)

array([[0.997, 1.   , 0.573, 0.   , 0.   ],
       [1.   , 1.   , 0.997, 0.   , 0.   ],
       [1.   , 1.   , 1.   , 0.997, 0.   ],
       [0.986, 0.995, 1.   , 1.   , 0.2  ],
       [1.   , 0.992, 1.   , 1.   , 1.   ]])